In [21]:
import pandas as pd
import requests

In [22]:
#---------------------------------------------------
#
# Config
#
#---------------------------------------------------

# This file should be automatically generated from "1. Gaze Projections and AOIs Mapping"
GAZE_DATA_WITH_AOIS_FILE = "data/gazeDataWithAOIs.csv"

In [23]:
#----------------------------------------------------------------------------------------
#
#
# 1. Oculomotor event detection (i.e., Fixations and saccades)
# 
#
#----------------------------------------------------------------------------------------

In [24]:
# Read gaze data with AOIs using pandas library
data = pd.read_csv(GAZE_DATA_WITH_AOIS_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [25]:
# Preview data
data.head()

,Row,Respondent,Age,Gender,Group,Calibration,Timestamp,EventSource,SlideEvent,StimType,Duration,CollectionPhase,SourceStimuliName,EventSource.1,ET_GazeLeftx,ET_GazeLefty,ET_GazeRightx,ET_GazeRighty,ET_PupilLeft,ET_PupilRight,ET_TimeSignal,ET_DistanceLeft,ET_DistanceRight,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,Chat,CTA_1,Date_Place_1,Date_Place_2,Description_1,Description_2,Fav,Food,Gen_Info,Ingredients,Instructions,Online,Participants_Overview,Participant_Bio,Participant_Profile_Picture,Place,Responsabilities,Status,Verified
0,1,P01,0,OTHER,Default,Excellent,112.2004,1.0,StartSlide,TestImage,300000.0,StimuliDisplay,Slide1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,P01,0,OTHER,Default,Excellent,113.2910,NaN,NaN,NaN,NaN,NaN,Slide1,1.0,1074.0,590.0,1081.0,527.0,3.323868,3.239807,149.999,657.255432,650.518372,0.568510,0.498617,0.419377,0.498766,0.0,0.0,1077.5,558.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,P01,0,OTHER,Default,Excellent,121.6164,NaN,NaN,NaN,NaN,NaN,Slide1,1.0,1080.0,579.0,1078.0,541.0,NaN,NaN,158.325,657.347839,650.507507,0.568528,0.498499,0.419368,0.498608,0.0,0.0,1079.0,560.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,P01,0,OTHER,Default,Excellent,129.9499,NaN,NaN,NaN,NaN,NaN,Slide1,1.0,1078.0,572.0,1082.0,559.0,NaN,NaN,166.658,657.481262,650.567383,0.568659,0.498408,0.419488,0.498481,0.0,0.0,1080.0,565.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,5,P01,0,OTHER,Default,Excellent,138.2936,NaN,NaN,NaN,NaN,NaN,Slide1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,175.002,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
#Keep only the gaze data coming from the eye-tracking device
data = data[ data['EventSource.1'] == 1 ]

In [27]:
#Fill n/a values with -1 (for JSON Compliance in requests) 
data = data.fillna(-1) 

In [28]:
# Oculomotor event detection (i.e., Fixations and saccades)
# Using customizable I-VT (Velocity Threshold) filter for fixation and saccade detection

dataWithEvents = None

# Iteratate over the participants and the apply I-VT filter on the data of each participant
for participant in data['Respondent'].unique():
    
    # filter in the participant data
    ParticipantData = data[data['Respondent']==participant]
    
    print(f'Applying fixation filter on the data for participant: {participant}')
    config = {
                    "fixationFilter" : "IVT",
                    "screenResolutionWidth" : 1920, 
                    "screenResolutionHeight" : 1080, 
                    "monitorSize" : 24, 
                    "screenDistance" : 60,
                    "gapFill" : True, 
                    "maxGapLength": 75, 
                    "noiseReduction" : False, 
                    "filterType" : "Median", 
                    "windowNoise" : 3,
                    "windowVelocity" : 20, 
                    "velocityThreshold" : 30, 
                    "mergeFixation" : True, 
                    "maxTimeBtwFixation" : 75,
                    "maxAngleBtwFixation" : 0.5, 
                    "discardShortFixation" : True,
                    "minDurationFixation" : 60
    }

    # POST pariticipant gaze data to the (R) eventDetection server
    url = 'http://rserver:6789/eventDetection'
    #url = 'http://127.0.0.1:6789/eventDetection'
    payload = {'config': config, 'data': ParticipantData.to_dict(orient="records")}
    response = requests.post(url, json = payload)
    
    # Create new a dataframe from the json response of the (R) eventDetection server
    participantDataWithEvents = pd.DataFrame.from_records(response.json())
    
    #Assign participantDataWithEvents to dataWithEvents or concate
    if dataWithEvents is None: 
        dataWithEvents = participantDataWithEvents
    else:
        dataWithEvents = pd.concat([dataWithEvents, participantDataWithEvents], axis=0)
        
    print('... Complete')

Applying fixation filter on the data for participant: P01
... Complete
Applying fixation filter on the data for participant: P02
... Complete
Applying fixation filter on the data for participant: P03
... Complete


In [29]:
# Preview gaze data with event information
dataWithEvents.head()

,Row,Respondent,Age,Gender,Group,Calibration,Timestamp,SourceStimuliName,EventSource.1,ET_PupilLeft,ET_PupilRight,ET_TimeSignal,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,Chat,CTA_1,Date_Place_1,Date_Place_2,Description_1,Description_2,Fav,Food,Gen_Info,Ingredients,Instructions,Online,Participants_Overview,Participant_Bio,Participant_Profile_Picture,Place,Responsabilities,Status,Verified,InterpolatedGazeX,InterpolatedGazeY,InterpolatedDistance,GazeVelocityAngle,FixID,Fixation Index by Stimulus,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,GazeAccelerationAngle,SacID,Saccade Index by Stimulus,Saccade Start,Saccade End,Saccade Duration,Saccade Amplitude,Saccade Peak Velocity,Saccade Peak Acceleration,Saccade Peak Deceleration,Saccade Direction
0,2,P01,0,OTHER,Default,Excellent,113.2910,Slide1,1,3.3239,3.2398,149.999,0.5685,0.4986,0.4194,0.4988,0,0,1077.5,558.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1077.5,558.5,653.8869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,P01,0,OTHER,Default,Excellent,121.6164,Slide1,1,NaN,NaN,158.325,0.5685,0.4985,0.4194,0.4986,0,0,1079.0,560.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1079.0,560.0,653.9277,10.8007,1.0,,1077.5797,563.1233,117.4537,375.7792,258.3255,0.2166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,P01,0,OTHER,Default,Excellent,129.9499,Slide1,1,NaN,NaN,166.658,0.5687,0.4984,0.4195,0.4985,0,0,1080.0,565.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1080.0,565.5,654.0243,10.5689,1.0,,1077.5797,563.1233,117.4537,375.7792,258.3255,0.2166,-27.8160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,P01,0,OTHER,Default,Excellent,138.2936,Slide1,1,NaN,NaN,175.002,NaN,NaN,NaN,NaN,4,4,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1077.0,567.0,654.0483,9.7254,1.0,,1077.5797,563.1233,117.4537,375.7792,258.3255,0.2166,-101.0827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,P01,0,OTHER,Default,Excellent,146.6364,Slide1,1,NaN,NaN,183.345,0.5686,0.4983,0.4195,0.4983,0,0,1074.0,568.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1074.0,568.5,654.0723,3.8549,1.0,,1077.5797,563.1233,117.4537,375.7792,258.3255,0.2166,-703.6661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# export dataWithEvents as csv
dataWithEvents.to_csv("data/gazeDataWithAOIsAndEvents.csv",  index=False)